In [1]:
import sagemaker
import numpy as np
import pandas as pd
import os
import torch
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel
from source_roberta import utils

tag2name = utils._get_tag2name()

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

## Deploy model

In [3]:
class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data='s3://sagemaker-eu-west-1-087816224558/sagemaker-pytorch-2020-06-06-17-15-12-540/output/model.tar.gz',
                     role = role,
                     framework_version='1.1.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)

In [4]:
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.m4.xlarge',
                         endpoint_name='ner-roberta-v12')

---------------!

In [5]:
print(f'Endpoint name: {predictor.endpoint}')

Endpoint name: ner-roberta-v12


## Test the endpoint on single input

In [6]:
def predict_on_text(text, predictor):
    """
    Method for computing predicitons on given text using predictor.
    Parameters:
    text       - single string
    predictor  - PyTorchPredictor object
    Returns: pandas dataframe with input words and predicted tags
    """
    predictions = predictor.predict(text)
    pred_array = np.fromstring(predictions[1:-1], dtype=np.int, sep=', ')
    input_length = len(str(text).split())
    df_preds = pd.DataFrame(zip([tag2name[i] for i in pred_array[1:(input_length+1)].tolist()], str(text).split()))
    df_preds.columns = ['Tag', 'Word']
    return df_preds

In [7]:
text = 'I live in USA this is my IBM laptop'
df_preds = predict_on_text(text, predictor)
print(df_preds)

     Tag    Word
0      O       I
1      O    live
2      O      in
3  B-geo     USA
4      O    this
5      O      is
6      O      my
7  B-org     IBM
8      O  laptop


### Create predictor from endpoint

In [ ]:
# from sagemaker.pytorch import PyTorchPredictor
# predictor = PyTorchPredictor('sagemaker-pytorch-2020-06-01-18-56-03-855',ContentType = 'text/plain')